In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [89]:
# Load training data
train_df_ori = pd.read_csv('train.csv')

# Load test data

test_df_ori = pd.read_csv('test.csv')

In [90]:
train_df_ori.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [91]:
test_df_ori.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [92]:
train_df_ori['HomePlanet'] = train_df_ori['HomePlanet'].fillna(train_df_ori['HomePlanet'].mode())
# train_df_ori['HomePlanet'] = train_df_ori['HomePlanet'].astype('category')
train_df_ori['CryoSleep'] = train_df_ori['CryoSleep'].fillna(train_df_ori['CryoSleep'].mode())
# train_df_ori['CryoSleep'] = train_df_ori['CryoSleep'].astype('category')
train_df_ori['Destination'] = train_df_ori['Destination'].fillna(train_df_ori['Destination'].mode())
# train_df_ori['Destination'] = train_df_ori['Destination'].astype('category')

train_df_ori["VIP"] = train_df_ori["VIP"].fillna(False).astype('category')

train_df_ori[['Deck', 'Num', 'Side']] = train_df_ori['Cabin'].str.split('/', expand=True)
train_df_ori['Deck'].fillna(0, inplace=True)
train_df_ori['Num'].fillna(0, inplace=True)
train_df_ori['Side'].fillna(0, inplace=True)

train_df_ori['Deck'] = train_df_ori['Deck'].astype('category')
train_df_ori['Side'] = train_df_ori['Side'].astype('category')
train_df_ori['Num'] = train_df_ori['Num'].astype('int')

train_df_ori.drop(columns=['Cabin', 'Name', 'PassengerId'], inplace=True)


/var/folders/76/jthg53_n64d8_v2ll16qt85r0000gp/T/ipykernel_54925/1146363303.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df_ori["VIP"] = train_df_ori["VIP"].fillna(False).astype('category')


In [93]:
train_df_ori.describe(include='all')

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side
count,8492,8476,8511,8514.000000,8693,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000,8693,8693,8693.000000,8693
unique,3,2,3,NaN,2,NaN,NaN,NaN,NaN,NaN,2,9,NaN,3
top,Earth,False,TRAPPIST-1e,NaN,False,NaN,NaN,NaN,NaN,NaN,True,F,NaN,S
freq,4602,5439,5915,NaN,8494,NaN,NaN,NaN,NaN,NaN,4378,2794,NaN,4288
mean,NaN,NaN,NaN,28.827930,NaN,224.687617,458.077203,173.729169,311.138778,304.854791,NaN,NaN,586.624065,NaN
std,NaN,NaN,NaN,14.489021,NaN,666.717663,1611.489240,604.696458,1136.705535,1145.717189,NaN,NaN,513.880084,NaN
min,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN
25%,NaN,NaN,NaN,19.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,152.000000,NaN
50%,NaN,NaN,NaN,27.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,407.000000,NaN
75%,NaN,NaN,NaN,38.000000,NaN,47.000000,76.000000,27.000000,59.000000,46.000000,NaN,NaN,983.000000,NaN


In [76]:
X_train, X_test, y_train, y_test = train_test_split(train_df_ori.drop(columns=['Transported']), train_df_ori['Transported'], test_size=.2)

# D_train = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
# D_test = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)


In [77]:
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', enable_categorical=True)
model.fit(X_train, y_train)

TypeError: object of type 'numpy.bool' has no len()

In [ ]:
from xgboost import XGBClassifier
# read data
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=.2)
# create model instance
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)

In [ ]:
preds

array([2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 0, 0, 0, 0, 1, 2, 1, 0, 2, 1, 0, 2,
       0, 2, 0, 2, 0, 1, 1, 1])